<a href="https://colab.research.google.com/github/AMX07/makemore/blob/main/makemore-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
# E06: meta-exercise! Think of a fun/interesting exercise and complete it.

In [ ]:
## E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?


#explore/read the dataset by storing the contents in variable of data structure list of strings

words = open('names.txt').read().splitlines()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
F.cross_entropy(input, target)

tensor(2.2813, grad_fn=<NllLossBackward0>)

tensor([4, 1, 4])

In [ ]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

NN approach
instead of having a look-up table which stores the 'counts',
we have weights which we get as a result of gradient descent.

how to get the weights? in another words how to train the model?
NN structure

xenc @ W

initiation
start with random weights
have a represenation of the alphabets which we can mathematically operate on.

forward pass
exnc @ W log-counts so logits

counts = logits.exp easier to operate mathematically?
prob = counts/counts.sum(1,keepdim=true)

loss = -prob[torch.arange(num),ys].log().mean()

backward pass


In [ ]:
chs = sorted(set("".join(words)))

stoi = {s:i+1 for i,s in enumerate(chs)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
#create dataset
x = []
y = []
for w in words:
  # print(w)
  chs = ['.'] + list(w) + ['.']

  for ch1, ch2 in zip(chs,chs[1:]):
    x.append(stoi[ch1])
    y.append(stoi[ch2])

x = torch.tensor(x)
y = torch.tensor(y)
num = x.nelement()
print(num)
xenc = F.one_hot(x,27).float()


# initate random W weights and use a generator with a seed to get the same results back!
g = torch.Generator().manual_seed(2147483647)
W = torch.rand((27,27), generator = g, requires_grad = True,dtype=torch.float)
# in this 27 by 27 matrix, we have
# so what xenc @ W reall does is that it selects a row of W, how do you select a row of W using indexing? W[x]
# so which rows should be selected? ith rows where i is in x.items()
#




228146


In [ ]:
for k in range(500):
  #forward
  logits = W[x]
  counts = logits.exp()
  probs = counts/counts.sum(1,keepdim=True)
  loss = - probs[torch.arange(num),y].log().mean()
  print(loss.item())
  #backward
  W.grad = None
  loss.backward()
  #update params
  W.data += -50 * W.grad

2.4564311504364014
2.4564263820648193
2.4564216136932373
2.4564168453216553
2.4564123153686523
2.456407308578491
2.456402540206909
2.4563980102539062
2.456393241882324
2.456388473510742
2.4563839435577393
2.4563794136047363
2.4563748836517334
2.4563701152801514
2.4563655853271484
2.4563610553741455
2.4563565254211426
2.4563519954681396
2.4563474655151367
2.456342935562134
2.4563381671905518
2.456333875656128
2.456329584121704
2.456325054168701
2.4563207626342773
2.4563162326812744
2.4563119411468506
2.4563076496124268
2.456303119659424
2.456299066543579
2.4562947750091553
2.4562904834747314
2.4562859535217285
2.456281900405884
2.456277370452881
2.456273317337036
2.4562692642211914
2.4562652111053467
2.4562606811523438
2.45625638961792
2.456252336502075
2.4562480449676514
2.4562439918518066
2.456239700317383
2.456235885620117
2.4562315940856934
2.4562275409698486
2.456223487854004
2.4562196731567383
2.4562156200408936
2.456211805343628
2.456207513809204
2.4562034606933594
2.456199407577

KeyboardInterrupt: 

In [ ]:
W.size() # 27,27
# W[torch.arange(2)]
logits = xenc @ W
logits.shape #228146, 27
l = x.tolist()
W[l]

tensor([[0.7081, 0.3542, 0.1054,  ..., 0.9928, 0.3419, 0.2070],
        [0.2629, 0.1921, 0.0504,  ..., 0.8833, 0.8975, 0.5053],
        [0.4098, 0.4807, 0.6182,  ..., 0.9564, 0.3926, 0.8961],
        ...,
        [0.7362, 0.3015, 0.2502,  ..., 0.9793, 0.3303, 0.8327],
        [0.2973, 0.0136, 0.3904,  ..., 0.4831, 0.3783, 0.8224],
        [0.4570, 0.9394, 0.9213,  ..., 0.0919, 0.2963, 0.9742]],
       grad_fn=<IndexBackward0>)

In [ ]:
logits

tensor([[0.7081, 0.3542, 0.1054,  ..., 0.9928, 0.3419, 0.2070],
        [0.2629, 0.1921, 0.0504,  ..., 0.8833, 0.8975, 0.5053],
        [0.4098, 0.4807, 0.6182,  ..., 0.9564, 0.3926, 0.8961],
        ...,
        [0.7362, 0.3015, 0.2502,  ..., 0.9793, 0.3303, 0.8327],
        [0.2973, 0.0136, 0.3904,  ..., 0.4831, 0.3783, 0.8224],
        [0.4570, 0.9394, 0.9213,  ..., 0.0919, 0.2963, 0.9742]],
       grad_fn=<MmBackward0>)

In [ ]:
for i in range(5):
  ix1 = 0
  ix2 = 0
  out = []
  while True:
    x1 = torch.tensor([ix1])
    x2 = torch.tensor([ix2])
    x1enc = F.one_hot(x1,num_classes=27)
    x2enc = F.one_hot(x2,num_classes=27)
    xenc = torch.cat((x1enc,x2enc),1).float()
    logits = xenc @ W
    counts = logits.exp()
    p = counts / counts.sum(1, keepdim=True)
    ix3 = (torch.multinomial(p, num_samples=1,replacement=True, generator=g)).item()
    out.append(itos[ix3])
    if ix3 == 0:
      break
    ix1 , ix2 = ix2, ix3
  print(''.join(out))

yandan.
zaylizap.
owrnayl.
ulee.
dezenleret.


HW-2, spliting datasets into dev, training and test!!

In [ ]:
n = torch.randperm(len(words))
n_train = int(0.8*len(words))
n_dev = int(0.1*len(words))

train = [words[i] for i in n[:n_train]]
dev = [words[i] for i in n[n_train:n_train+n_dev]]
test = [words[i] for i in n[n_train+n_dev:]]


In [ ]:
## E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model
#- i.e. try many possibilities and see which one works best based on the dev set loss.
#What patterns can you see in the train and dev set loss as you tune this strength?
#Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?


In [ ]:
#traing a trigram model with regularization


# gotta create dataset!
x1 = []
x2 = []
y  = []

for w in train:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs,chs[1:],chs[2:]):
    x1.append(stoi[ch1])
    x2.append(stoi[ch2])
    y.append(stoi[ch3])

x1 = torch.tensor(x1) #dtype is int
x2 = torch.tensor(x2)
# do we need y to be a tensor? Yes!!
y = torch.tensor(y)
n1 = x1.nelement()
print(x2.nelement())
# do we need n2? nope



# one_hot_encoding for representations!
x1enc = F.one_hot(x1,27)
x2enc = F.one_hot(x2,27)

#gotta concat them!
xenc = torch.cat((x1enc,x2enc),dim=1).float() # shape 156807 x 54


# initate W and g

g = torch.Generator().manual_seed(214783647)


156912


In [ ]:

# evaluate the wights on dev


# gotta create dataset!
x1_dev = []
x2_dev= []
y_dev  = []

for w in dev:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs,chs[1:],chs[2:]):
    x1_dev.append(stoi[ch1])
    x2_dev.append(stoi[ch2])
    y_dev.append(stoi[ch3])

x1_dev = torch.tensor(x1_dev) #dtype is int
x2_dev = torch.tensor(x2_dev)
y_dev = torch.tensor(y_dev)
n_dev = x1_dev.nelement()
print(n_dev)



# one_hot_encoding for representations!
x1enc_dev = F.one_hot(x1_dev,27)
x2enc_dev = F.one_hot(x2_dev,27)

#gotta concat them!
xenc_dev = torch.cat((x1enc_dev,x2enc_dev),dim=1).float() # shape 156807 x 54

# logits_dev = xenc @ W
# counts_dev = logits_dev .exp()
# probs_dev = counts_dev/counts_dev.sum(dim=1,keepdim=True)
# loss_dev = - probs_dev[torch.arange(n1),y].log().mean() + 0.000001 * (W**2).mean()

#if I increase the regularization strenght the loss increase, if I decrease the loss decrease
# W = torch.rand((54,27),generator=g,dtype=torch.float,requires_grad=True)
# logits_dev = xenc_dev @ W
# counts_dev = logits_dev.exp()
# probs_dev = counts_dev/counts_dev.sum(dim=1,keepdim=True)
# loss_dev = - probs_dev[torch.arange(n_dev),y_dev].log().mean()


19645


In [ ]:
# Training the model
for reg_st in [1,0.1,0.001,0.0001,0.000001,0.0000001,0.00000001]:
  W = torch.rand((54,27),generator=g,dtype=torch.float,requires_grad=True)
  print("reg_st loss_train loss_dev")
  for k in range(500):
    # forward pass
    logits = xenc @ W
    counts = logits.exp()
    probs = counts/counts.sum(dim=1,keepdim=True)
    loss = - probs[torch.arange(n1),y].log().mean() + reg_st * (W**2).mean()
    # print(loss.data)

    #backward pass
    W.grad = None
    loss.backward()

    #update the parameters
    W.data += - 50 * W.grad

    #final loss approx 2.2529



  logits_dev = xenc_dev @ W
  counts_dev = logits_dev.exp()
  probs_dev = counts_dev/counts_dev.sum(dim=1,keepdim=True)
  loss_dev = - probs_dev[torch.arange(n_dev),y_dev].log().mean()
  print(reg_st, loss,loss_dev)



reg_st loss_train loss_dev
1 tensor(2.5453, grad_fn=<AddBackward0>) tensor(2.3901, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
0.1 tensor(2.3189, grad_fn=<AddBackward0>) tensor(2.2616, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
0.001 tensor(2.2419, grad_fn=<AddBackward0>) tensor(2.2364, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
0.0001 tensor(2.2404, grad_fn=<AddBackward0>) tensor(2.2364, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
1e-06 tensor(2.2403, grad_fn=<AddBackward0>) tensor(2.2363, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
1e-07 tensor(2.2402, grad_fn=<AddBackward0>) tensor(2.2363, grad_fn=<NegBackward0>)
reg_st loss_train loss_dev
1e-08 tensor(2.2402, grad_fn=<AddBackward0>) tensor(2.2363, grad_fn=<NegBackward0>)


In [ ]:
#0.0000001

In [ ]:
for i in range(5):
  out = []
  x1 = 0
  x2 = 0
  while True:
    # print(out)
    x1enc = F.one_hot(torch.tensor([x1]),num_classes=27)
    x2enc = F.one_hot(torch.tensor([x2]),num_classes=27)
    xenc = torch.cat((x1enc,x2enc),dim=1).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts/counts.sum(dim=1,keepdim=True)
    x3 = torch.multinomial(probs,num_samples=1, replacement=True,generator=g).item()
    if x3 == 0:
      break
    out.append(itos[x3])
    x1,x2 = x2,x3
  print("".join(out))


halanleo
novin
evamikarien
yanjezien
ialifanna


19592


/tmp/ipython-input-1860358920.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x1_dev = torch.tensor(x1) #dtype is int
/tmp/ipython-input-1860358920.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x2_dev = torch.tensor(x2)


tensor(2.6682, grad_fn=<AddBackward0>)

In [ ]:
# evaluate the wights on test



# gotta create dataset!
x1 = []
x2 = []
y  = []

for w in test:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs,chs[1:],chs[2:]):
    x1.append(stoi[ch1])
    x2.append(stoi[ch2])
    y.append(stoi[ch3])

x1 = torch.tensor(x1) #dtype is int
x2 = torch.tensor(x2)
# do we need y to be a tensor? nope
n1 = x1.nelement()
print(x2.nelement())
# do we need n2? nope



# one_hot_encoding for representations!
x1enc = F.one_hot(x1,27)
x2enc = F.one_hot(x2,27)

#gotta concat them!
xenc = torch.cat((x1enc,x2enc),dim=1).float() # shape 156807 x 54

logits = xenc @ W
counts = logits.exp()
probs = counts/counts.sum(dim=1,keepdim=True)
#
loss = - probs[torch.arange(n1),y].log().mean() + 0.0000001* (W**2).mean()

#if I increase the regularization strenght the loss increase, if I decrease the loss decrease

loss

19556


tensor(2.2486, grad_fn=<AddBackward0>)